In [2]:
import pandas as pd
import numpy as np
import sys
import math
import scipy

In [3]:
def check_numeric(data_col, threshold):
    """
    Check numeric
    :param data_col: one data column from file csv.
    :param threshold: the point at which you start evaluate the result is NG,NA or OK.
    :return: result (OK, NG, NA), k (numbers of numeric rows), N (total values),
    abnormal_data_details (DataFrame contains values and indexes of invalid numeric).
     """
    #create dictionnary to constains values and indexes of invalid numeric
    abnormal_data_details = {'Indexes': [], 'Values': []}

    indexes = [] #indexes of invalid numeric
    values = [] #values of invalid numeric
    
    #for loop to fill data to 2 list (indexes and values)
    for index, value in data_col.iteritems():
        value = str(value)
        row_isnumeric = value.isnumeric()
        if not row_isnumeric:
            indexes.append(index)
            values.append(value)

    #update value in dict and change into data frame
    abnormal_data_details['Indexes'] = indexes
    abnormal_data_details['Values'] = values
    abnormal_data_details = pd.DataFrame(abnormal_data_details)

    result = 'NA'
    k = len(data_col) - len(values)  # number of numeric rows: total_row - abnormal_row
    N = len(data_col)  # total values
    #Condition to get result by the ratio of k/N and theshold
    ratio = k/N
    if k/N >= threshold and k < N:
        result = 'NG'
    elif k == N:
        result = 'OK'
    
    #write to file
    
    with open('SummaryOutput.txt','w') as f:
        f.write(str(k/N))
        f.close()

    return result, k, N, abnormal_data_details

In [4]:

def mean(data_col):
    return (sum(data_col) / len(data_col))


def stdev(data):
    mu = mean(data)
    return math.sqrt(sum([(point - mu) ** 2 for point in data]) / len(data))


def check_value_range(data_col, threshold_range_z_score, threshold_range_iqr):
    """
    The function checks value range of the data column
    :param data_col: one data column from file csv.
    :param threshold_range_z_score: threshold_z_score of the length function
    :param threshold_range_iqr: threshold_z_iqr of the length function
    :return: result 'OK', 'NA', 'NG'
    """
    # set result's default as 'NA'
    result = 'NA'
    # numbers of outliers in data_col
    outliers = []

    # check numeric in data_col
    for row in data_col:
        # If the column has a value isn't numeric, return 'NA'
        row = str(row)
        if not row.isnumeric():
            return result

    # If the column has 100% value is numeric
    # convert into integer
    for i in range(len(data_col)):
        data_col[i] = int(data_col[i])
    # sort data_col
    data_col_sorted = np.sort(data_col)

    # get the middle value's index
    median_index = len(data_col_sorted) // 2 if len(data_col_sorted) % 2 == 0 else len(data_col_sorted) // 2 + 1

    # If length of data_col is even number
    # First quartile Q1 is value calculate from index 0 to median_index
    # Third quartile Q3 is value calculate from median_index+1 to index -1
    if len(data_col_sorted) % 2 == 0:
        Q1 = np.median(data_col_sorted[:median_index])
        Q3 = np.median(data_col_sorted[median_index:])

        # If length of data_col is odd number
    # First quartile Q1 is value calculate from index 0 to median_index-1
    # Third quartile Q3 is value calculate from median_index+1 to index -1
    else:
        Q1 = np.median(data_col_sorted[:median_index - 1])
        Q3 = np.median(data_col_sorted[median_index:])

    # Calculate IQR
    IQR = Q3 - Q1

    # Calculate [Low, Up] value based on threshold_iqr:
    Low = Q1 - threshold_range_iqr * IQR
    Up = Q3 + threshold_range_iqr * IQR
    # If x value is greater than Up or smaller than Lower, x is outlier.
    for x in data_col:
        if x > Up or x < Low:
            outliers.append(x)  # if x is outlier, append x to outliers[]

    # Initialization
    # Calculate mean of all values in column:
    Vmean = mean(data_col)

    # Calculate the standard deviation:
    Vstdev = stdev(data_col)

    # Calculate z-score of each value Vi: Vi_zscore = (Vi - Vmean) / Vstdev
    # If abs(Vi_zscore)> threshold_z_score, x is outlier.
    for Vi in data_col:
        Vi_zscore = (Vi - Vmean) / Vstdev
        if abs(Vi_zscore) > threshold_range_z_score:
            outliers.append(Vi)  # if Vi is outlier, append Vi to outliers[]

    if len(outliers) > 0:
        result = 'NG'
    elif len(outliers) == 0:
        result = 'OK'

    return result

In [5]:

def check_length(data_col, threshold_length):
    """
    The function checks the length of the data column.
    :param data_col: data column.
    :param threshold_length: threshold of the length function.
    :return: result 'OK', 'NA', 'NG'.
    """

    # set default result as 'NA'
    result = 'NA'
    # get len of data_col as N
    N = len(data_col)
    # Setting up a new value chain is the length of each value in the original string
    Li = []
    for li in data_col:
        Li.append(len(str(li)))  # add length of each value in data_col
    list_value = {str(i): Li.count(i) for i in Li}
    max_ki = max(list_value.values())
    if max_ki / N >= threshold_length and max_ki / N < 1:
        result = 'NG'
    elif max_ki / N == 1:
        result = 'OK'

    return result, max_ki, len(data_col)

In [6]:

def check_category(data_col, threshold_category):
    """
    The function checks category of the data column
    :param data_col:  one data column from file csv.
    :param threshold_category: threshold of the length function.
    :return: result 'OK', 'NA', 'NG'.
    """

    Pi = []
    result = 'NA'
    outliers = []
    set_data_col = set(data_col)
    # for loop uses to count the frequence of each unique value
    for i in set_data_col:
        count = 0  # reset count varialbe for each value
        for j in data_col:
            if j == i:
                count += 1
        Pi.append(count)  # add number of each unique value to Pi

    Pmean = mean(Pi)
    Pstdev = stdev(Pi)

    for pi in Pi:
        Li_zscore = (pi - Pmean) / Pstdev
        # if Li_zscore < thresholdLength then it is outlier,
        if Li_zscore < threshold_category:
            outliers.append(Li_zscore)  # append Li_zscore to outliers[]

    if len(outliers) == 0:
        result = 'OK'
    elif len(outliers) > 0:
        result = 'NG'

    return result, outliers

In [17]:
df = pd.read_csv('DBDetectOutlier.csv', dtype='object')

In [18]:
for col in df.columns:
    if type(col) != int() or type(col) != float():
        print('Column"s Name: {}'.format(col))
        print(check_numeric(df[col], 0.95))

Column"s Name: BANKCode
('NG', 41, 42,    Indexes Values
0        0    11O)
Column"s Name: KEIY_SEQ
('OK', 42, 42, Empty DataFrame
Columns: [Indexes, Values]
Index: [])
Column"s Name: CIF_NO
('NA', 7, 42,     Indexes Values
0         7    nan
1         8    nan
2         9    nan
3        10    nan
4        11    nan
5        12    nan
6        13    nan
7        14    nan
8        15    nan
9        16    nan
10       17    nan
11       18    nan
12       19    nan
13       20    nan
14       21    nan
15       22    nan
16       23    nan
17       24    nan
18       25    nan
19       26    nan
20       27    nan
21       28    nan
22       29    nan
23       30    nan
24       31    nan
25       32    nan
26       33    nan
27       34    nan
28       35    nan
29       36    nan
30       37    nan
31       38    nan
32       39    nan
33       40    nan
34       41    nan)
Column"s Name: LOGIN_PWD
('NA', 0, 42,     Indexes                            Values
0         0  4BACA5C5A939

In [8]:
for col in df.columns:
    if type(col) != int() or type(col) != float() :
         print("Column's Name: {}".format(col))
         print(check_value_range(df[col], 2, 1))


Column's Name: BANKCode
NA
Column's Name: KEIY_SEQ
OK
Column's Name: CIF_NO
NA
Column's Name: LOGIN_PWD
NA
Column's Name: LOGIN_PWD_Date
NA
Column's Name: User_PWD
NA
Column's Name: CreateDate
NA
Column's Name: Email
NA
Column's Name: STRT_YMD
OK
Column's Name: STOP_YMD
NA
Column's Name: T_DT
NA
Column's Name: MAIL_STOP_DT
NA


In [9]:
for col in df.columns:
    if type(col) != int() or type(col) != float() :
        print("Column's Name: {}".format(col))
        print(check_length(df[col], 0.95))

Column's Name: BANKCode
('OK', 42, 42)
Column's Name: KEIY_SEQ
('NG', 41, 42)
Column's Name: CIF_NO
('NA', 35, 42)
Column's Name: LOGIN_PWD
('NA', 23, 42)
Column's Name: LOGIN_PWD_Date
('NA', 34, 42)
Column's Name: User_PWD
('NA', 39, 42)
Column's Name: CreateDate
('NA', 31, 42)
Column's Name: Email
('NG', 40, 42)
Column's Name: STRT_YMD
('OK', 42, 42)
Column's Name: STOP_YMD
('NA', 32, 42)
Column's Name: T_DT
('NA', 39, 42)
Column's Name: MAIL_STOP_DT
('NA', 29, 42)


In [19]:
for col in df.columns:
    if type(col) != int() or type(col) != float() :
        print("Column's Name: {}".format(col))
        print(check_category(df[col], 0.84))

Column's Name: BANKCode
('NG', [0.24576957615571215, 0.08192319205190406, -1.556540648986177])
Column's Name: KEIY_SEQ
('NG', [-0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192, -0.1581138830084192])
Column's Name: CIF_NO
('NG', [-1